In [1]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../Dataset/df_final_oficial_concatenado_com_referencias.csv")

/tmp/ipykernel_3839094/2947361084.py:1: DtypeWarning: Columns (3,4,5,7,8,11,19,37,43,48,51,53,59,60,61,69) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Dataset/df_final_oficial_concatenado_com_referencias.csv")


In [88]:
drops = ['DOI','Abstract','Cited References']
df_dropped = df.dropna(subset=drops)
df_dropped = df_dropped.head(10)

In [110]:
def articles_to_dict(df,remove_unknow_articles:bool=False):
    dictionary = {}
    index = df.index
    df_dois = "DOI " + df['DOI'].str.upper()
    df_dois_list = list(df_dois)
    max_index = index[-1]
    for i in index:
        print(f"Progresso:{i} de {max_index}",end='\r')
        cited_references = df['Cited References'][i]
        collected_dois = re.findall(r'DOI [^;]+;', cited_references)
        
        collected_dois_correction = []
        for j in collected_dois:
            doi = j.removesuffix(";").upper()
            if "DOI [" in doi:
                doi_list_str = doi.removeprefix("DOI [").replace(']','').replace(',','').split()                
                doi = f"DOI {doi_list_str[0]}"
                
            if remove_unknow_articles:
                if doi in df_dois_list:
                    collected_dois_correction.append(doi)
                else:
                    continue
                
            if not remove_unknow_articles:
                collected_dois_correction.append(doi)
        
        doi_key = df_dois[i].upper()
        dictionary[doi_key] = collected_dois_correction
        
    return dictionary


def find_groups(graph,dictionary:dict,save_csv:bool=False,save_name='any',
                print_progress:bool=False,n_step_progress:int=1):
   
    connections = list(nx.connected_components(graph))
    groups_dictionary = {}
    all_cited_articles = list(dictionary.keys())

    if print_progress:
        print("Processo inicial feito")


    for i,group in enumerate(connections):
        articles = []
        for j,article in enumerate(group):
            if article in all_cited_articles:
                articles.append(article)
            if print_progress and (j+1)%n_step_progress == 0:
                print(f"Progresso: {j+1} de {len(group)}, em {i+1} de {len(connections)}",end='\r')
        if print_progress:
            print(f"\n Tamanho do grupo:{len(articles)}")
        groups_dictionary[f"Grupo {i+1}"] = articles

    if save_csv:
        print("Saving dataset")
        df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in groups_dictionary.items()]))
        df.to_csv(f'{save_name}.csv', index=False, encoding='utf-8')


    return groups_dictionary
   

In [111]:
dicionario = articles_to_dict(df_dropped,True)
grafo = nx.Graph(dicionario)

In [112]:
grupos = find_groups(grafo,dicionario,True,"teste",True,1)

Processo inicial feito
Progresso: 2 de 2, em 1 de 9
 Tamanho do grupo:2
Progresso: 1 de 1, em 2 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 3 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 4 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 5 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 6 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 7 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 8 de 9
 Tamanho do grupo:1
Progresso: 1 de 1, em 9 de 9
 Tamanho do grupo:1
Saving dataset


In [91]:
def plot_graph(graph,dictionary):

    isolated_nodes = set(graph.nodes) - set(dictionary.keys())

    pos = nx.spring_layout(graph, seed=20)

    # nós isolados (vermelhos)
    nx.draw_networkx_nodes(graph, pos, nodelist=isolated_nodes, node_color='red', node_size=50, edgecolors='black',
                        label="Artigos fora da amostra")

    # nós conectados (azuis)
    nx.draw_networkx_nodes(graph, pos, nodelist=set(graph.nodes) - isolated_nodes, node_color='skyblue', node_size=50, edgecolors='black',
                        label="Artigos presentes na amostra")

    nx.draw_networkx_edges(graph, pos)

    plt.axis("off") 
    plt.legend()
    plt.show()
    
    
    
def encontrar_grupos_isolados(dictionary,graph):
    
    componentes = list(nx.connected_components(graph))

    pos = nx.spring_layout(graph, seed=20)
    colors = [
    'red', 'blue', 'green', 'orange', 'purple', 'pink', 'cyan', 'yellow',
    'brown', 'gray', 'black', 'lime', 'magenta', 'teal', 'navy',
    'gold', 'indigo', 'violet', 'turquoise', 'salmon', 'maroon',
    'olive', 'chocolate', 'orchid', 'plum', 'slateblue', 'crimson', 'darkgreen',
    'darkblue', 'darkred', 'darkorange', 'deeppink', 'deepskyblue', 'dodgerblue',
    'greenyellow', 'lightblue', 'lightcoral', 'lightgreen', 'lightpink',
    'lightsalmon', 'lightseagreen', 'lightskyblue', 'lime']
    plt.figure(figsize=(8, 6))

    for i, grupo in enumerate(componentes):
        nx.draw_networkx_nodes(graph, pos, nodelist=grupo, node_color=colors[i % len(colors)], label=f'Grupo {i+1}', node_size=200)

    nx.draw_networkx_edges(graph, pos, alpha=0.5)
    
    isolated_nodes = set(graph.nodes) - set(dictionary.keys())


    nx.draw_networkx_nodes(graph, pos, nodelist=isolated_nodes, node_color='red', node_size=20, edgecolors='black',
                        label="Artigos fora da amostra")

    nx.draw_networkx_nodes(graph, pos, nodelist=set(graph.nodes) - isolated_nodes, node_color='skyblue', node_size=20, edgecolors='black',
                        label="Artigos presentes na amostra")
    nx.draw_networkx_edges(graph, pos)
    
    #plt.legend()
    plt.show()
    


In [96]:
#plot_graph(grafo,dicionario)

In [95]:
#encontrar_grupos_isolados(dicionario,grafo)